In [2]:
!pip install neat-python
!pip install graphviz

/bin/bash: /home/marcel/anaconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 kB 2.2 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [25]:
import warnings

import graphviz
import matplotlib.pyplot as plt
import numpy as np


def plot_stats(statistics, ylog=False, view=True, filename='avg_fitness.svg'):
    """ Plots the population's average and best fitness. """
    if plt is None:
        warnings.warn("This display is not available due to a missing optional dependency (matplotlib)")
        return

    generation = range(len(statistics.most_fit_genomes))
    best_fitness = [c.fitness for c in statistics.most_fit_genomes]
    avg_fitness = np.array(statistics.get_fitness_mean())
    stdev_fitness = np.array(statistics.get_fitness_stdev())

    plt.plot(generation, avg_fitness, 'b-', label="average")
    plt.plot(generation, avg_fitness - stdev_fitness, 'g-.', label="-1 sd")
    plt.plot(generation, avg_fitness + stdev_fitness, 'g-.', label="+1 sd")
    plt.plot(generation, best_fitness, 'r-', label="best")

    plt.title("Population's average and best fitness")
    plt.xlabel("Generations")
    plt.ylabel("Fitness")
    plt.grid()
    plt.legend(loc="best")
    if ylog:
        plt.gca().set_yscale('symlog')

    #plt.savefig(filename)
    if view:
        plt.show()

    plt.close()


def plot_spikes(spikes, view=False, filename=None, title=None):
    """ Plots the trains for a single spiking neuron. """
    t_values = [t for t, I, v, u, f in spikes]
    v_values = [v for t, I, v, u, f in spikes]
    u_values = [u for t, I, v, u, f in spikes]
    I_values = [I for t, I, v, u, f in spikes]
    f_values = [f for t, I, v, u, f in spikes]

    fig = plt.figure()
    plt.subplot(4, 1, 1)
    plt.ylabel("Potential (mv)")
    plt.xlabel("Time (in ms)")
    plt.grid()
    plt.plot(t_values, v_values, "g-")

    if title is None:
        plt.title("Izhikevich's spiking neuron model")
    else:
        plt.title("Izhikevich's spiking neuron model ({0!s})".format(title))

    plt.subplot(4, 1, 2)
    plt.ylabel("Fired")
    plt.xlabel("Time (in ms)")
    plt.grid()
    plt.plot(t_values, f_values, "r-")

    plt.subplot(4, 1, 3)
    plt.ylabel("Recovery (u)")
    plt.xlabel("Time (in ms)")
    plt.grid()
    plt.plot(t_values, u_values, "r-")

    plt.subplot(4, 1, 4)
    plt.ylabel("Current (I)")
    plt.xlabel("Time (in ms)")
    plt.grid()
    plt.plot(t_values, I_values, "r-o")

    if filename is not None:
        plt.savefig(filename)

    if view:
        plt.show()
        plt.close()
        fig = None

    return fig


def plot_species(statistics, view=False, filename='speciation.svg'):
    """ Visualizes speciation throughout evolution. """
    if plt is None:
        warnings.warn("This display is not available due to a missing optional dependency (matplotlib)")
        return

    species_sizes = statistics.get_species_sizes()
    num_generations = len(species_sizes)
    curves = np.array(species_sizes).T

    fig, ax = plt.subplots()
    ax.stackplot(range(num_generations), *curves)

    plt.title("Speciation")
    plt.ylabel("Size per Species")
    plt.xlabel("Generations")

    plt.savefig(filename)

    if view:
        plt.show()

    plt.close()


def draw_net(config, genome, view=False, filename=None, node_names=None, show_disabled=True, prune_unused=False,
             node_colors=None, fmt='svg'):
    """ Receives a genome and draws a neural network with arbitrary topology. """
    # Attributes for network nodes.
    if graphviz is None:
        warnings.warn("This display is not available due to a missing optional dependency (graphviz)")
        return

    # If requested, use a copy of the genome which omits all components that won't affect the output.
    if prune_unused:
        genome = genome.get_pruned_copy(config.genome_config)

    if node_names is None:
        node_names = {}

    assert type(node_names) is dict

    if node_colors is None:
        node_colors = {}

    assert type(node_colors) is dict

    node_attrs = {
        'shape': 'circle',
        'fontsize': '9',
        'height': '0.2',
        'width': '0.2'}

    dot = graphviz.Digraph(format=fmt, node_attr=node_attrs)

    inputs = set()
    for k in config.genome_config.input_keys:
        inputs.add(k)
        name = node_names.get(k, str(k))
        input_attrs = {'style': 'filled', 'shape': 'box', 'fillcolor': node_colors.get(k, 'lightgray')}
        dot.node(name, _attributes=input_attrs)

    outputs = set()
    for k in config.genome_config.output_keys:
        outputs.add(k)
        name = node_names.get(k, str(k))
        node_attrs = {'style': 'filled', 'fillcolor': node_colors.get(k, 'lightblue')}

        dot.node(name, _attributes=node_attrs)

    used_nodes = set(genome.nodes.keys())
    for n in used_nodes:
        if n in inputs or n in outputs:
            continue

        attrs = {'style': 'filled',
                 'fillcolor': node_colors.get(n, 'white')}
        dot.node(str(n), _attributes=attrs)

    for cg in genome.connections.values():
        if cg.enabled or show_disabled:
            # if cg.input not in used_nodes or cg.output not in used_nodes:
            #    continue
            input, output = cg.key
            a = node_names.get(input, str(input))
            b = node_names.get(output, str(output))
            style = 'solid' if cg.enabled else 'dotted'
            color = 'green' if cg.weight > 0 else 'red'
            width = str(0.1 + abs(cg.weight / 5.0))
            dot.edge(a, b, _attributes={'style': style, 'color': color, 'penwidth': width})

    dot.render(filename, view=view)

    return dot

In [26]:
"""
2-input XOR example -- this is most likely the simplest possible example.
"""

import os

import neat
import visualize

# 2-input XOR inputs and expected outputs.
xor_inputs = [(0.0, 0.0), (0.0, 1.0), (1.0, 0.0), (1.0, 1.0)]
xor_outputs = [(0.0,), (1.0,), (1.0,), (0.0,)]


def eval_genomes(genomes, config):
    for genome_id, genome in genomes:
        genome.fitness = 4.0
        net = neat.nn.FeedForwardNetwork.create(genome, config)
        for xi, xo in zip(xor_inputs, xor_outputs):
            output = net.activate(xi)
            genome.fitness -= (output[0] - xo[0]) ** 2


def run(config_file):
    # Load configuration.
    config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                         neat.DefaultSpeciesSet, neat.DefaultStagnation,
                         config_file)

    # Create the population, which is the top-level object for a NEAT run.
    p = neat.Population(config)

    # Add a stdout reporter to show progress in the terminal.
    p.add_reporter(neat.StdOutReporter(True))
    stats = neat.StatisticsReporter()
    p.add_reporter(stats)
    p.add_reporter(neat.Checkpointer(5))

    # Run for up to 300 generations.
    winner = p.run(eval_genomes, 300)

    # Display the winning genome.
    print('\nBest genome:\n{!s}'.format(winner))

    # Show output of the most fit genome against training data.
    print('\nOutput:')
    winner_net = neat.nn.FeedForwardNetwork.create(winner, config)
    for xi, xo in zip(xor_inputs, xor_outputs):
        output = winner_net.activate(xi)
        print("input {!r}, expected output {!r}, got {!r}".format(xi, xo, output))

    node_names = {-1: 'A', -2: 'B', 0: 'A XOR B'}
    draw_net(config, winner, True, node_names=node_names)
    draw_net(config, winner, True, node_names=node_names, prune_unused=False)
    plot_stats(stats, ylog=False, view=True)
    plot_species(stats, view=True)

    p = neat.Checkpointer.restore_checkpoint('neat-checkpoint-4')
    p.run(eval_genomes, 10)



In [27]:
local_dir = os.path.dirname("neat/")
config_path = os.path.join(local_dir, 'neat_config_xor.txt')
run(config_path)


 ****** Running generation 0 ****** 

Population's average fitness: 2.21044 stdev: 0.35196
Best fitness: 2.98847 - size: (1, 2) - species 1 - id 130
Average adjusted fitness: 0.557
Mean genetic distance 1.125, standard deviation 0.420
Population of 150 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    0   150      3.0    0.557     0
Total extinctions: 0
Generation time: 0.012 sec

 ****** Running generation 1 ****** 

Population's average fitness: 2.34769 stdev: 0.31685
Best fitness: 2.99898 - size: (1, 2) - species 1 - id 239
Average adjusted fitness: 0.538
Mean genetic distance 1.196, standard deviation 0.454
Population of 150 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    1   150      3.0    0.538     0
Total extinctions: 0
Generation time: 0.009 sec (0.010 average)

 ****** Running generation 2 ****** 

Population's average fitness: 2.32749 stdev: 


 ****** Running generation 30 ****** 

Population's average fitness: 2.49400 stdev: 0.51883
Best fitness: 3.40171 - size: (2, 4) - species 3 - id 4368
Average adjusted fitness: 0.530
Mean genetic distance 2.535, standard deviation 0.938
Population of 150 members in 4 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1   30    31      3.0    0.601     9
     2   17    65      3.0    0.654     0
     3    8    38      3.4    0.497     1
     4    1    16      2.6    0.370     0
Total extinctions: 0
Generation time: 0.014 sec (0.014 average)

 ****** Running generation 31 ****** 

Population's average fitness: 2.43459 stdev: 0.49034
Best fitness: 3.45682 - size: (2, 4) - species 3 - id 4675
Average adjusted fitness: 0.482
Mean genetic distance 2.626, standard deviation 0.950
Population of 151 members in 5 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1   31    18      3.0    0.489    10
  

    10    2    30      3.0    0.426     1
    11    1     5      3.0    0.295     0
Total extinctions: 0
Generation time: 0.020 sec (0.018 average)

 ****** Running generation 53 ****** 

Population's average fitness: 2.58216 stdev: 0.51001
Best fitness: 3.76994 - size: (2, 5) - species 3 - id 6901
Average adjusted fitness: 0.528
Mean genetic distance 3.080, standard deviation 0.878
Population of 150 members in 11 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     2   40    18      3.0    0.577    19
     3   31     9      3.8    0.547     6
     4   24     8      2.0    0.344     1
     5   22    11      3.0    0.424    10
     6   15    14      3.0    0.555    14
     7   13    16      3.0    0.620     8
     8   10    17      3.0    0.586     8
     9    5    18      3.0    0.640     3
    10    3    21      3.0    0.566     0
    11    2    10      3.0    0.421     1
    12    0     8       --       --     0
Total extinctions: 0
Genera

Mean genetic distance 3.406, standard deviation 1.232
Population of 149 members in 9 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     3   50    27      3.8    0.283    25
     4   43    16      3.8    0.461    16
    10   22    19      3.0    0.329    19
    11   21    17      3.0    0.490     0
    12   19    18      3.0    0.340    17
    13   10    13      3.0    0.187     4
    14    6     8      2.6    0.062     0
    15    5    24      3.0    0.330     3
    16    2     7      2.0    0.030     1
Total extinctions: 0
Generation time: 0.018 sec (0.017 average)

 ****** Running generation 73 ****** 

Population's average fitness: 2.42179 stdev: 0.49967
Best fitness: 3.78575 - size: (2, 5) - species 4 - id 8095

Species 10 with 19 members is stagnated: removing it
Average adjusted fitness: 0.256
Mean genetic distance 3.369, standard deviation 1.234
Population of 150 members in 8 species:
   ID   age  size  fitness  adj fit  stag
  ====

/tmp/ipykernel_55077/4014237103.py:34: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
/tmp/ipykernel_55077/4014237103.py:108: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


Mean genetic distance 1.284, standard deviation 0.487
Mean genetic distance 1.279, standard deviation 0.534
Mean genetic distance 1.394, standard deviation 0.529
Mean genetic distance 1.348, standard deviation 0.412
Mean genetic distance 1.434, standard deviation 0.481
Mean genetic distance 1.359, standard deviation 0.449
Mean genetic distance 1.409, standard deviation 0.449
Mean genetic distance 1.730, standard deviation 0.417
Mean genetic distance 1.891, standard deviation 0.510
Mean genetic distance 1.808, standard deviation 0.484
